# Analysis of Short Term Airbnb rentals #
By comparing and contrasting short term rentals between Seattle and Boston, we hope to gain a deeper understanding of the price market than we would by looking at just one city. We'll define a short term rental as any rental that can be rented as one week or less (Note: This is a little stricter than the definition frequently used by governments of one month, but more reasonable matches the bulk of the Airbnb data). We'll look to answer:
1) Can we accurately predict prices of each city? Is this easier for one city (or does the same method not work for both)?
2) What are the most important factors in common to both cities?
3) Are there factors that are unique to one city or the other?

### Data Gathering ###

First, import any dependencies

In [343]:
import pandas as pd
import helperFunctions
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import normalize
import math

Next, grab the data we're working with. We'll be using both the Boston and Seattle listing data that Airbnb provided.

In [344]:
dfBL = pd.read_csv('../data/boston/listings.csv')
dfSL = pd.read_csv('../data/seattle/listings.csv')

We want to compare and contrast our models, so we want to run them on similar parameters. The Boston data actually has a few more columns than the Seattle data, so we'll remove them. Seattle doesn't have any columns that Boston doesn't.

In [345]:
BL = set(dfBL.columns)
SL = set(dfSL.columns)
print(f"Boston only cols: {BL.difference(SL)}")
print(f"Seattle only cols: {SL.difference(BL)}")
dfBL.drop(BL.difference(SL),axis=1,inplace=True)

Boston only cols: {'access', 'interaction', 'house_rules'}
Seattle only cols: set()


We also want to focus our analysis on short term rentals, so we'll remove any rentals that require more than a week of renting.

In [346]:
dfBL.drop(dfBL[dfBL['minimum_nights'] > 7].index,inplace=True)
dfSL.drop(dfSL[dfSL['minimum_nights'] > 7].index,inplace=True)

We should take a look at which columns have missing values:

In [347]:
for column in dfBL.columns:
    if dfBL[column].isnull().any():
        print(f"{column} : {dfBL[column].isnull().sum()}")

summary : 138
space : 999
neighborhood_overview : 1342
notes : 1882
transit : 1216
thumbnail_url : 525
medium_url : 525
xl_picture_url : 525
host_location : 10
host_about : 1234
host_response_time : 450
host_response_rate : 450
host_acceptance_rate : 450
host_neighbourhood : 333
neighbourhood : 514
neighbourhood_group_cleansed : 3402
city : 2
zipcode : 36
market : 13
property_type : 3
bathrooms : 14
bedrooms : 10
beds : 9
square_feet : 3349
weekly_price : 2554
monthly_price : 2582
security_deposit : 2131
cleaning_fee : 1053
has_availability : 3402
first_review : 681
last_review : 681
review_scores_rating : 733
review_scores_accuracy : 743
review_scores_cleanliness : 738
review_scores_checkin : 740
review_scores_communication : 738
review_scores_location : 742
review_scores_value : 741
license : 3402
jurisdiction_names : 3402
reviews_per_month : 681


In [348]:
for column in dfSL.columns:
    if dfSL[column].isnull().any():
        print(f"{column} : {dfSL[column].isnull().sum()}")

summary : 173
space : 560
neighborhood_overview : 1016
notes : 1576
transit : 917
thumbnail_url : 303
medium_url : 303
xl_picture_url : 303
host_name : 2
host_since : 2
host_location : 8
host_about : 843
host_response_time : 513
host_response_rate : 513
host_acceptance_rate : 754
host_is_superhost : 2
host_thumbnail_url : 2
host_picture_url : 2
host_neighbourhood : 295
host_listings_count : 2
host_total_listings_count : 2
host_verifications : 2
host_has_profile_pic : 2
host_identity_verified : 2
neighbourhood : 411
zipcode : 7
property_type : 1
bathrooms : 16
bedrooms : 6
beds : 1
square_feet : 3671
weekly_price : 1789
monthly_price : 2282
security_deposit : 1936
cleaning_fee : 1016
first_review : 609
last_review : 609
review_scores_rating : 629
review_scores_accuracy : 640
review_scores_cleanliness : 635
review_scores_checkin : 640
review_scores_communication : 633
review_scores_location : 637
review_scores_value : 638
license : 3768
reviews_per_month : 609


There sure is a lot of columns with missing values! Fortunately, not all of this is bad news. Some of these columns will end up being "Is there data or is there not"? A URL is meaningless to our mode for instance, but just the fact that there's *something* isn't. Some of this is also data that if present, is redundant or close to redundant for what we're trying to predit (weekly and monthly price), so that's not something we want to include in our model. Some of this will also be cut down as we cut down on redundant columns (neighborhood and its variants will be dropped in favor of zip codes).

This being said, there's a chunk of columns where nans will have to be either immuted with something or dropped. In general, of the columns we'll end up keeping, if we immute it will be immuted with 0 or the mode. Security deposit, cleaning fees, fees for extra people, and reviews per month all fill this. They all represent columns where no data is implicitly 0. If the home owner doesn't have a fee listed, that can also be intrepreted as a fee of $0. Through experimentation, mean immutation was found to either not have a beneficial effect over the mean, or plummet the effectiveness of the model (Seattle in particular was more sensitive to this than Boston)

Columns where drop rows in the columns are missing are trickier. Frequently, the Boston data was more robust to immutation. It was found that anything review related was best left dropped, as well as date columns so as "host since", which suggested some sort of data error to begin with.

### Data Cleaning ###

These columns are either meaningless (i.e. - id, urls that are always present), about the dataset (scraping, url), can't be trusted to be accurate (latitude/longitude), redudant to other data (a significant amount of the location ones), beyond the scope of what we're trying to do here (anything that would require image processing or NLP), or is subtlely just actually the price. We want to predict the price from the house, not from the price! Other columns are just missing too much data -square feet for instance, to be reliable. Instead, bedrooms and bathrooms does a good job standing in.

In [349]:
drop_list = ['id','last_scraped', 'listing_url', 'scrape_id','thumbnail_url',
               'medium_url', 'host_id', 'host_url','host_name','city',
               'state','market','country', 'latitude', 'longitude',
               'calendar_last_scraped','is_location_exact','smart_location',
               'requires_license','license','picture_url','square_feet',
               'monthly_price','weekly_price','jurisdiction_names',
               'neighbourhood_group_cleansed','experiences_offered',
               'calendar_updated','country_code','calendar_updated',
               'host_thumbnail_url','host_picture_url','has_availability',
               'host_location','description','name','street','neighbourhood',
               'host_neighbourhood','neighbourhood_cleansed','host_listings_count',
               'host_has_profile_pic']

These columns have some sort of enumeration to them

In [350]:
dummy_cols = ['cancellation_policy','bed_type','room_type',
                 'property_type','host_response_time','zipcode']
listed_enum_cols = ['amenities','host_verifications']

Various cleaning. Some cells lack data, which is implictly zero (if there's no cost for having extra people, that cost is $0.00).  Some columns we want to remove the data that's incomplete, some we only care if there's data or there's not data, and some we need to convert booleans. For data in which missing data was removed, experimentation was used to determine this to be the best path. Injecting the mean was considered, but plummetted the ability of the model to make prediction. With bathrooms, bedrooms, and beds in the mix this should make sense. It's possible for something like "review_scores_location", a better method could be done via injecting some association between the column's values and the various zip codes they are associated with. This could be explored as a future path.

In [377]:
is_null_cols = ['xl_picture_url','transit', 'host_about',
                  'notes','neighborhood_overview','space','summary']
immute_zero_cols = ['security_deposit','cleaning_fee','extra_people','reviews_per_month']
immute_mode_cols = ['bathrooms','bedrooms','beds']
immute_mean_cols = []
remove_nan_cols = ['host_identity_verified',
                  'host_total_listings_count',
                  'host_since_month','host_since_day','host_since_year',
                  'first_review_day','first_review_month','first_review_year',
                  'last_review_day','last_review_month','last_review_year',
                  'review_scores_rating','review_scores_accuracy',
                  'review_scores_cleanliness','review_scores_checkin',
                  'review_scores_communication','review_scores_location',
                  'review_scores_value','host_acceptance_rate','host_response_rate']
bool_cols = ['host_is_superhost','host_identity_verified',
                'instant_bookable','require_guest_profile_picture','require_guest_phone_verification']

### Data Modelling ###

The following function cleans the data and returns a linear model, the training data, test data, and r2 scores.

In [378]:
def createLM(df_in, test_size=.3, random_state=42):
    '''
    INPUT:
    df_in - Input data frame
    test_size - Determines ratio of split of test/train data
    random_state - Seed value for repeatable randomization
    OUTPUT:
    r2_scores_test - R2 score for test data
    r2_scores_train - R2 score for train data
    lm_model - The created linear model
    X_train - The training input data
    X_test - The testing input data
    y_train - The training output data
    y_test - The testing output data
    X_reduced - The normalized input data
    
    This function takes in one cities data frame, converts data types to numeric data where applicable,
    boolean data where applicable, and immutes and removes data where applicable, and creates a linear model.
    '''
    df = df_in.copy(deep=True)
    df.drop(drop_list,axis=1,inplace=True)
    helperFunctions.convert_date_to_numeric(df)
    helperFunctions.convert_dollars_to_numeric(df)
    helperFunctions.convert_percent_to_numeric(df)
    for col in immute_zero_cols:
        df[col].fillna(0,inplace=True)
    for col in immute_mode_cols:
        df[col].fillna(df[col].mode()[0],inplace=True)
    for col in immute_mean_cols:
        df[col].fillna(df[col].mean(),inplace=True)
    for col in remove_nan_cols:
        df.dropna(subset=[col],inplace=True)
    for col in bool_cols:
        helperFunctions.convert_column_to_bool(df, col)
    #Create dummy columns
    for col in dummy_cols:
        df = pd.get_dummies(df, columns=[col], drop_first=True, dummy_na=True)
    for cat in listed_enum_cols:
        df = helperFunctions.complex_category_to_dummy(df, cat)
        df.drop(cat,axis=1,inplace=True)
    for col in is_null_cols:
        df[f"{col}_null"] = df[col].isnull()
        df.drop([col],axis=1,inplace=True)
    y = df['price']
    X = df.drop(['price'],axis=1)
    x_cols = X.columns
    #Remove columns that have no variance. These don't contribute to the model and can lead to some weird behavior
    for col in x_cols:
        if len(X[col].unique())<2:
            X.drop([col],axis=1,inplace=True)
    
    for column in X.columns:
        if X[column].isnull().any():
            print(f"{column} : {X[column].isnull().sum()}")
        
    # Normalize data so inference can be made, create training and test data
    # Note scikit learn normalization is sample based normalization so complete data set can be done all at once
    X_norm = normalize(X)
    X_train, X_test, y_train, y_test = train_test_split(
        X_norm, y, test_size=test_size, random_state=random_state)

    # fit the model
    lm_model = LinearRegression()
    lm_model.fit(X_train, y_train)
    y_test_preds = lm_model.predict(X_test)
    y_train_preds = lm_model.predict(X_train)
    r2_scores_test = r2_score(y_test, y_test_preds)
    r2_scores_train = r2_score(y_train, y_train_preds)
    return (r2_scores_test, r2_scores_train, lm_model, X_train, X_test, y_train, y_test,X)

In [379]:
r2_scores_test_bl, r2_scores_train_bl, lm_model_bl, X_train_bl, X_test_bl, y_train_bl, y_test_bl,X_original = createLM(dfBL)
r2_scores_test_sl, r2_scores_train_sl, lm_model_sl, X_train_sl, X_test_sl, y_train_sl, y_test_sl,X_original = createLM(dfSL)

### How well can we predict price in each city? ###

In [380]:
print(f"Boston R2 test vs train: {r2_scores_test_bl}, {r2_scores_train_bl}")
print(f"Seattle R2 test vs train: {r2_scores_test_sl}, {r2_scores_train_sl}")

Boston R2 test vs train: 0.6640950254236129, 0.7318537462086685
Seattle R2 test vs train: 0.5943311118712487, 0.7270869298547613


These results suggest some overfitting (moreso for Seattle than Boston), and while we can use the same methedology for both cities, we perform better in Boston than Seattle, suggesting difficulty in future generalizations for this process if this approach was to be taken across more cities.

### What's the most important factors for price in each city? ###

Finally, we can inspect some our model. We're interested in what the biggest impact factors for both cities are, so we can compare and contrast. We'll also look at which factor had the biggest positive impact.

It's worth noting that are model for Seattle appears to be taking the location and using that to negatively impact the price. This doesn't mean that every location is bad for your price - just relative to the "base location", which happened to be the column removed from the set when creating the dummy variables. It does mean it won't appear in our "most positive" impact coefficient. A good example at why looking at the absolute coefficient and just determining the magnitude of impact a feature has on the variance of the model can be more valuable than looking at the raw coefficient.

In [381]:
df_coef_bl = helperFunctions.coef_weights(lm_model_bl, reduce_X_bl)
df_coef_bl.head(30)

,est_int,coefs,abs_coefs
110,"""Washer / Dryer""",1.130971e+06,1.130971e+06
58,property_type_Other,-5.363202e+05,5.363202e+05
103,zipcode_02467,-4.874537e+05,4.874537e+05
86,zipcode_02132,-4.663784e+05,4.663784e+05
93,zipcode_02141,-4.210164e+05,4.210164e+05
85,zipcode_02131,-3.785242e+05,3.785242e+05
77,zipcode_02122,-3.623884e+05,3.623884e+05
90,zipcode_02136,-3.573482e+05,3.573482e+05
101,zipcode_02445,-3.425200e+05,3.425200e+05
79,zipcode_02125,-3.418227e+05,3.418227e+05


In [382]:
df_coef_sl = helperFunctions.coef_weights(lm_model_sl, reduce_X_sl)
df_coef_sl.head(30)

,est_int,coefs,abs_coefs
50,property_type_Boat,1.608703e+06,1.608703e+06
56,property_type_Dorm,-3.604528e+05,3.604528e+05
91,zipcode_98178,-3.345220e+05,3.345220e+05
48,room_type_Shared room,-2.476108e+05,2.476108e+05
85,zipcode_98133,-2.338921e+05,2.338921e+05
73,zipcode_98108,-1.891401e+05,1.891401e+05
79,zipcode_98118,-1.755192e+05,1.755192e+05
71,zipcode_98106,-1.742208e+05,1.742208e+05
90,zipcode_98177,-1.711820e+05,1.711820e+05
83,zipcode_98125,-1.466094e+05,1.466094e+05


In [383]:
df_coef_sl.loc[df_coef_sl['coefs']==df_coef_sl['coefs'].max()]

,est_int,coefs,abs_coefs
50,property_type_Boat,1.608703e+06,1.608703e+06


In [384]:
df_coef_bl.loc[df_coef_bl['coefs']==df_coef_bl['coefs'].max()]

,est_int,coefs,abs_coefs
110,"""Washer / Dryer""",1.130971e+06,1.130971e+06


This rough analysis highlights non-zipcode properties that both Seattle and Boston have in common as significant factors. Because the zip codes themselves are dummy variables that are unique to each city, they won't be shared.

In [390]:
df_coef_bl_no_zip = df_coef_bl.loc[df_coef_bl['est_int'].str.contains('zipcode')==False]
df_coef_sl_no_zip = df_coef_sl.loc[df_coef_sl['est_int'].str.contains('zipcode')==False]
boston_set = set(df_coef_bl_no_zip['est_int'].head(30).unique())
seattle_set = set(df_coef_sl_no_zip['est_int'].head(30).unique())
print(f"Intersection of significant coefficients: {boston_set.intersection(seattle_set)}")

Intersection of significant coefficients: {'bathrooms', "'sent_id'", "'manual_offline'", 'property_type_Dorm', "'manual_online'", 'require_guest_profile_picture', 'room_type_Shared room', 'property_type_Boat', 'host_is_superhost', "'weibo'", 'bedrooms', 'room_type_Private room', "'phone'"}


### Finally, are there unique factors beyond the location for each city? ###

By ignoring the zipcodes and other common significant parameters in our models' coefficients, we'll look for anything truly unexpected. We can look at the list of top factors and see what stands out as unique between Seattle and Boston.

In [393]:
df_coef_sl_unique = df_coef_sl.loc[df_coef_sl['est_int'].str.contains('zipcode')==False]
df_coef_sl_unique = df_coef_sl_unique.loc[df_coef_sl_unique['est_int'].str.contains('room_type')==False]
df_coef_sl_unique = df_coef_sl_unique.loc[df_coef_sl_unique['est_int'].str.contains('property_type')==False]
df_coef_sl_unique = df_coef_sl_unique.loc[df_coef_sl_unique['est_int'].str.contains('bedrooms')==False]
df_coef_sl_unique = df_coef_sl_unique.loc[df_coef_sl_unique['est_int'].str.contains('bathrooms')==False]
df_coef_sl_unique = df_coef_sl_unique.loc[df_coef_sl_unique['est_int'].str.contains('weibo')==False]
df_coef_sl_unique = df_coef_sl_unique.loc[df_coef_sl_unique['est_int'].str.contains('phone')==False]
df_coef_sl_unique = df_coef_sl_unique.loc[df_coef_sl_unique['est_int'].str.contains('manual_o')==False]
df_coef_sl_unique.head(10)

,est_int,coefs,abs_coefs
122,Doorman,107309.270052,107309.270052
104,"""Wireless Internet""",105402.986463,105402.986463
123,Internet,-74660.840551,74660.840551
147,'sent_id',-65708.751650,65708.751650
126,"""Hot Tub""",53729.386138,53729.386138
119,Pool,-48178.816807,48178.816807
127,Washer,-46903.938584,46903.938584
66,host_response_time_within an hour,-46633.371514,46633.371514
96,"""Smoking Allowed""",-41028.302950,41028.302950
37,require_guest_profile_picture,37341.254332,37341.254332


In [394]:
df_coef_bl_unique = df_coef_bl.loc[df_coef_bl['est_int'].str.contains('zipcode')==False]
df_coef_bl_unique = df_coef_bl_unique.loc[df_coef_bl_unique['est_int'].str.contains('room_type')==False]
df_coef_bl_unique = df_coef_bl_unique.loc[df_coef_bl_unique['est_int'].str.contains('property_type')==False]
df_coef_bl_unique = df_coef_bl_unique.loc[df_coef_bl_unique['est_int'].str.contains('bedrooms')==False]
df_coef_bl_unique = df_coef_bl_unique.loc[df_coef_bl_unique['est_int'].str.contains('bathrooms')==False]
df_coef_bl_unique = df_coef_bl_unique.loc[df_coef_bl_unique['est_int'].str.contains('weibo')==False]
df_coef_bl_unique = df_coef_bl_unique.loc[df_coef_bl_unique['est_int'].str.contains('phone')==False]
df_coef_bl_unique = df_coef_bl_unique.loc[df_coef_bl_unique['est_int'].str.contains('manual_o')==False]
df_coef_bl_unique.head(10)

,est_int,coefs,abs_coefs
110,"""Washer / Dryer""",1.130971e+06,1.130971e+06
43,cancellation_policy_super_strict_30,2.118770e+05,2.118770e+05
44,bed_type_Couch,1.574893e+05,1.574893e+05
160,'sent_id',-1.425814e+05,1.425814e+05
47,bed_type_Real Bed,1.281314e+05,1.281314e+05
111,"""Other pet(s)""",-9.007612e+04,9.007612e+04
133,"""Paid Parking Off Premises""",7.640499e+04,7.640499e+04
45,bed_type_Futon,7.231169e+04,7.231169e+04
46,bed_type_Pull-out Sofa,6.897468e+04,6.897468e+04
5,host_is_superhost,6.833679e+04,6.833679e+04
